# Tax-Calculator Demo
This notebook provides a demo of the `Tax-Calculator` micro-simulation model.  The goal is to show some of the inputs and outputs of this model to provide and understanding of what the foundation is for the tax functions that are used in the OG model.

#### Getting Started
To use this notebook, you will need the `Tax-Calculator` package (in addition to a number of packages that will come standard with the Anaconda distribution of Python).  To get `Tax-Calculator`, open your Anaconda command prompt (Windows) or terminal window (Mac).  From here, type

```
conda install -c ospc taxcalc
```

In [1]:
# imports
from taxcalc import *
from bokeh.io import show, output_notebook, save, output_file
output_notebook()

Loading BokehJS ...

In [19]:
# Initiate baseline calculator
rec = Records.cps_constructor()
pol = Policy()
calc1 = Calculator(policy=pol, records=rec)
calc1.advance_to_year(2018)
calc1.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


## Specifying a reform policy

The baseline in `Tax-Calculator` is a current-law baseline.  You can specify your reforms in a JSON file and, using the read_json_param_objects method, create a dictionary containing both policy reforms and behavioral assumptions
The policy reform and any behavioral assumptions need to be in separate JSON files. Save the policy reforms and the behvaioral assumptions and call them whatever you want and update the snippet below.

In [20]:
reforms = calc1.read_json_param_objects('2017_law.json', None)
reforms

{'behavior': {},
 'consumption': {},
 'growdiff_baseline': {},
 'growdiff_response': {},
 'policy': {2017: {'_ID_Medical_frt': [0.1], '_cpi_offset': [0.0025]},
  2018: {'_ACTC_Income_thd': [3000.0],
   '_ALD_DomesticProduction_hc': [0.0],
   '_CTC_c': [1000.0],
   '_CTC_ps': [[75000.0, 110000.0, 55000.0, 75000.0, 75000.0]],
   '_DependentCredit_Child_c': [0.0],
   '_DependentCredit_Nonchild_c': [0.0],
   '_DependentCredit_before_CTC': [False],
   '_ID_AllTaxes_c': [[9e+99, 9e+99, 9e+99, 9e+99, 9e+99]],
   '_ID_Casualty_hc': [0.0],
   '_ID_Charity_crt_all': [0.5],
   '_ID_Miscellaneous_hc': [0.0],
   '_ID_crt': [0.8],
   '_ID_prt': [0.03],
   '_II_rt1': [0.1],
   '_II_rt2': [0.15],
   '_II_rt3': [0.25],
   '_II_rt4': [0.28],
   '_II_rt5': [0.33],
   '_II_rt6': [0.35],
   '_II_rt7': [0.396],
   '_PT_excl_rt': [0.0],
   '_PT_excl_wagelim_prt': [[0.0, 0.0, 0.0, 0.0, 0.0]],
   '_PT_excl_wagelim_rt': [9e+99],
   '_PT_excl_wagelim_thd': [[0.0, 0.0, 0.0, 0.0, 0.0]],
   '_PT_rt1': [0.1],
   '_P

In [21]:
# Inititate calculator to apply reforms to
recs2 = Records.cps_constructor()
calc2 = Calculator(policy=pol, records=recs2)
pol.implement_reform(reforms['policy'])
calc2.advance_to_year(2018)
calc2.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


### Behavioral response

A dictionary is also used to implement behavioral reforms. The only difference is you must then pass the baseline and reform calculators into the response method of the behavioral class. This method calculates the change in tax liabilities and then, using the specified elasticities, returns a new calculator object that accounts for any behavioral change.

In [15]:
calc2.behavior.update_behavior(reforms['behavior'])
calc_response = Behavior.response(calc1, calc2)

AttributeError: 'function' object has no attribute 'update_behavior'

### Viewing the results

You can use built in methods to get MTR and ATR plots. Each one is returned as a simple Bokeh figure that you can then add to as desired

In [25]:
mtr_plot_data = calc1.mtr_graph(calc2, mars=2,
                                mtr_measure='itax',
                                mtr_variable='e00200p',
                                alt_e00200p_text='Wages and Salaries',
                                mtr_wrt_full_compen=False,
                                income_measure='agi',
                                dollar_weighting=False)
output_file(filename='wage_mtrs.html')
save(mtr_plot_data)
show(mtr_plot_data)
# mtr_plot_data = mtr_graph_data(calc1, calc2)
# mtr_plot_figure = xtr_graph_plot(mtr_plot_data, title=' ', ylabel='Marginal tax rate')
# #save plot
# output_file(filename='mtr_plot.html')
# save(mtr_plot_figure)
# show(mtr_plot_figure)

In [26]:
mtr_plot_data = calc1.mtr_graph(calc2, mars=2,
                                mtr_measure='itax',
                                mtr_variable='e00900p', 
                                alt_e00200p_text='Proprietorship Income',
                                mtr_wrt_full_compen=False,
                                income_measure='agi',
                                dollar_weighting=False)
output_file(filename='SchC_mtrs.html')
save(mtr_plot_data)
show(mtr_plot_data)

### Moving around over the budget window

The above plots were for 2018, the first year we ran the model for.  We can move the model forward in the budget window and look at outcomes in a different year.

In [28]:
calc1.advance_to_year(2026)
calc2.advance_to_year(2026)

In [29]:
mtr_plot_data = calc1.mtr_graph(calc2, mars=2,
                                mtr_measure='itax',
                                mtr_variable='e00900p', 
                                alt_e00200p_text='Proprietorship Income',
                                mtr_wrt_full_compen=False,
                                income_measure='agi',
                                dollar_weighting=False)
output_file(filename='SchC_mtrs_2026.html')
save(mtr_plot_data)
show(mtr_plot_data)

In [34]:
# Plot the average tax rate
atr_plot_data = calc1.atr_graph(calc2, mars=2,
                                atr_measure='combined')
output_file(filename='ATRs_2026.html')
save(atr_plot_data)
show(atr_plot_data)

### Multiyear difference table
The multiyear differences table function displays the difference between your baseline and refoms. You can group the results by weighted decile, decile, income bin, or age. The default is income bin.

In [39]:
calc1.diagnostic_table(num_years=2)

,2026,2027
Returns (#m),189.1,191.7
AGI ($b),"15,281.6","15,990.2"
Itemizers (#m),83.5,85.4
Itemized Deduction ($b),"2,360.0","2,492.2"
Standard Deduction Filers (#m),85.5,85.9
Standard Deduction ($b),911.1,933.4
Personal Exemption ($b),"1,601.1","1,656.9"
Taxable Income ($b),"10,843.7","11,353.6"
Regular Tax ($b),"2,071.1","2,171.8"
AMT Income ($b),"13,926.7","14,558.6"


In [44]:
dist1, dist2 = calc1.distribution_tables(calc2, 'weighted_deciles')
dist1

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,...,othertaxes,refund,iitax,payrolltax,combined,ubi,benefit_cost_total,benefit_value_total,expanded_income,aftertax_income
0-10n,"100,970","-34,483,875,342",0,"1,103,712,605",0,0,"928,872,093",0,0,"-34,483,875,342",...,0,"213,678","-213,678","66,836,174","66,622,496",0,"1,241,061,608","1,241,061,608","-33,162,328,710","-33,228,951,206"
0-10z,"1,339,234","-22,552,220",0,"13,951,900,749",0,0,"11,495,517,973",0,0,"-22,552,220",...,0,0,0,0,0,0,0,0,0,0
0-10p,"17,473,114","94,067,205,850","14,396,628","119,219,894,676","352,796","4,003,256,341","80,826,491,313","6,077,502,319","477,420,836","91,765,913,191",...,0,"10,290,297,346","-9,958,933,595","12,579,195,723","2,620,262,128",0,"44,352,846,734","44,352,846,734","139,122,294,647","136,502,032,519"
10-20,"18,913,183","247,213,558,165","11,388,283","146,737,103,315","3,033,088","37,547,517,808","129,510,890,898","64,760,374,369","6,368,084,942","223,712,516,396",...,0,"19,979,581,940","-14,638,367,933","35,212,966,577","20,574,598,644",0,"184,040,561,349","184,040,561,349","444,858,872,444","424,284,273,800"
20-30,"18,913,200","406,268,888,794","11,312,598","143,761,828,168","4,413,084","64,950,249,411","145,437,904,621","159,523,329,414","17,989,273,111","365,730,495,836",...,0,"20,380,018,042","-4,704,870,554","58,625,921,138","53,921,050,584",0,"232,627,344,952","232,627,344,952","669,611,927,744","615,690,877,160"
30-40,"18,912,968","559,008,118,705","10,983,975","142,545,814,010","5,446,331","87,048,743,292","158,723,703,741","265,728,620,041","31,961,995,969","504,491,070,732",...,0,"15,970,273,004","12,815,877,670","79,505,900,464","92,321,778,134",0,"292,044,501,505","292,044,501,505","893,370,344,438","801,048,566,304"
40-50,"18,914,362","770,412,476,539","10,126,267","135,749,026,825","7,039,941","127,819,404,228","175,102,477,339","409,238,600,295","51,166,378,735","690,704,777,700",...,0,"11,517,337,688","34,992,632,534","106,325,184,883","141,317,817,418",0,"335,507,012,471","335,507,012,471","1,154,968,697,167","1,013,650,879,749"
50-60,"18,913,366","1,041,879,561,865","8,654,083","122,566,212,393","8,883,280","187,188,876,081","189,845,568,354","606,759,936,486","82,817,405,160","926,171,622,068",...,0,"7,223,147,359","69,502,368,076","138,787,215,316","208,289,583,392",0,"385,826,989,511","385,826,989,511","1,483,396,951,797","1,275,107,368,405"
60-70,"18,911,948","1,405,768,506,657","7,263,750","106,717,373,852","10,694,192","262,780,345,093","203,484,533,574","883,698,866,043","129,928,868,080","1,245,376,608,195",...,0,"4,116,382,376","118,653,758,413","179,804,280,603","298,458,039,017",0,"447,353,908,746","447,353,908,746","1,916,604,109,161","1,618,146,070,144"
70-80,"18,914,581","1,886,162,701,543","5,577,070","86,866,061,849","12,541,469","351,384,768,766","216,875,531,705","1,271,024,936,918","193,416,910,093","1,677,097,186,356",...,0,"2,090,742,761","185,208,360,180","235,706,779,498","420,915,139,677",0,"531,718,724,669","531,718,724,669","2,503,731,043,129","2,082,815,903,452"
